In [ ]:
print("--- Step 0: Create Simulator with Observed Data and Learn Summary Statistics ---")


# Load model from YAML configuration
config_name = 'gauss_gauss_1d_default'
print(f"\nLoading model from: {config_name}")

# Correction du chemin relatif pour exécution depuis abcnre/examples/notebooks
config_path = Path("../../configs/models") / f"{config_name}.yml"
if config_path.exists():
    with open(config_path, "r") as f:
        config_model = yaml.safe_load(f)
else :
    raise FileNotFoundError(f"Configuration file {config_path} does not exist.")

gauss_model = _create_model_from_config(config=config_model, source_path=config_path)
print(f"Model loaded: {gauss_model}")

# Generate observed data FIRST
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)
true_theta = 2.5
epsilon_quantile = 1.
observed_data = gauss_model.simulate(subkey, theta=true_theta)
print(f"Observed data shape: {observed_data.shape}")

# Re-import ABCSimulator with updated code
from abcnre.simulation.simulator import ABCSimulator

# Create NEW simulator WITH observed data (important for consistent dimensions)
simulator = ABCSimulator(
    model=gauss_model,
    observed_data=observed_data,
    quantile_distance=epsilon_quantile
)
print(f"Simulator created with epsilon = {simulator.epsilon:.4f}")

# Import config_learner from external YAML file - corrected path
config_learner_path = Path("../../../src/abcnre/preprocessing/configs/summaryDeepSet.yaml")
with open(config_learner_path, "r") as f:
    config_learner = yaml.safe_load(f)

# Learn summary statistics using the simulator
key, key_learn = jax.random.split(key)
simulator.learn_summary_stats(key=key_learn, config_learner=config_learner)
si
print("✅ Summary statistics learned with DeepSet for i.i.d. data")

--- Step 0: Create Simulator with Observed Data and Learn Summary Statistics ---

Loading model from: gauss_gauss_1d_default


NameError: name '_create_model_from_config' is not defined

In [ ]:
print('--- Step 0.5: Verification of the learned summary statistics ---')

key, key_sample = jax.random.split(key)
n_samples = 10000

thetas, xs = simulator.model.sample_theta_x_multiple(key_sample, n_samples=n_samples)
statistics = simulator.summary_stats_fn(xs)

means = np.mean(xs, axis=1)
correlation = np.corrcoef(statistics.flatten(), means.flatten())[0, 1]
print(f"Correlations between summary statistics and means: {correlation}")
print(f"✅ Summary statistics are working correctly with shape: {statistics.shape}")

--- Step 0.5: Verification of the learned summary statistics ---
Correlations between summary statistics and means: 0.9992531905272054
✅ Summary statistics are working correctly with shape: (10000, 1)


In [ ]:
# --- Step 1: Display Simulator Configuration ---
print("--- Step 1: Display Simulator Configuration ---")

# The simulator was already created in Step 0 with proper dimensions
print(f"Using existing simulator with epsilon = {simulator.epsilon:.4f}")
print(f"Observed data shape: {simulator.observed_data.shape}")
print(f"Summary statistics enabled: {simulator.config.get('summary_stats_enabled', False)}")

# Display model properties from metadata
model_args = gauss_model.get_model_args()
print("\nModel arguments:", model_args)

--- Step 1: Display Simulator Configuration ---
Using existing simulator with epsilon = inf
Observed data shape: (10,)
Summary statistics enabled: True

Model arguments: {'model_type': 'GaussGaussModel', 'model_class': 'GaussGaussModel', 'model_args': {'mu0': 0.0, 'sigma0': 2.0, 'sigma': 0.5, 'n_obs': 10}, 'metadata': {'parameter_space_dim': 1, 'analytical_posterior': True, 'module': 'abcnre.simulation.models.gauss_gauss'}}


In [ ]:
# --- Step 1.5: Save Model Configuration to YAML ---
print("\--- Step 1.5: Save Model Configuration to YAML ---")

# Save the model configuration for reproducibility
model_yaml_path = output_dir / "model_config.yml"
save_model_to_yaml(gauss_model, model_yaml_path)
print(f"Model configuration saved to: {model_yaml_path}")

# Verify we can reload it
reloaded_model = create_model_from_yaml(model_yaml_path)
print(f"✅ Model successfully reloaded: {reloaded_model}")
print(f"Parameters match: {reloaded_model.mu0 == gauss_model.mu0 and reloaded_model.sigma0 == gauss_model.sigma0 and reloaded_model.sigma == gauss_model.sigma}")


--- Step 1.5: Save Model Configuration to YAML ---
Model configuration saved to: ../results/model_config.yml
✅ Model successfully reloaded: GaussGaussModel(mu0=0.0, sigma0=2.0, sigma=0.5)
Parameters match: True


In [ ]:
# --- Step 2: Save Simulator Configuration ---
print("--- Step 2: Save Simulator Configuration ---")
output_dir.mkdir(exist_ok=True, parents=True)
simulator.save(output_dir / "gauss_1D_simulator.yml")


--- Step 2: Save Simulator Configuration ---
✅ Simulator saved with hash: d1312ced323a
   - Configuration: ../results/gauss_1D_simulator.yml
   - Observed Data: ../results/observed_data_eps_inf_d1312ced323a.npy


In [ ]:
# --- Step 2.5: Demonstrate YAML Model Configuration ---
print("\n--- Step 2.5: Demonstrate YAML Model Configuration ---")

# Show the complete model configuration
print("Current model configuration:")
model_config = gauss_model.get_model_args()

print("Model configuration:")
for key_arg, value in model_config.items():
    print(f"  {key_arg}: {value}")


--- Step 2.5: Demonstrate YAML Model Configuration ---
Current model configuration:
Model configuration:
  model_type: GaussGaussModel
  model_class: GaussGaussModel
  model_args: {'mu0': 0.0, 'sigma0': 2.0, 'sigma': 0.5, 'n_obs': 10}
  metadata: {'parameter_space_dim': 1, 'analytical_posterior': True, 'module': 'abcnre.simulation.models.gauss_gauss'}


In [ ]:
# --- Step 3: Create Network and Training Configuration ---

print("\n--- Step 3: Create Network Configuration ---")
nre_config = get_nre_config('hybrid_deepset_reduce_on_plateau')  # Utiliser une configuration existante
nre_config.training.num_epochs = 50
nre_config.training.n_samples_per_epoch = 10240
nre_config.training.num_thetas_to_store = 10000

nre_config.save(network_config_path)
print(f"Training configuration saved to: {network_config_path}")
print(f"Using configuration: {nre_config.experiment_name}")
print(f"Network type: {nre_config.network.network_type}")
print(f"LR Schedule: {nre_config.training.lr_scheduler.schedule_name}")


--- Step 3: Create Network Configuration ---
Training configuration saved to: ../results/config_mlp_reduce_on_plateau.yml
Using configuration: hybrid_deepset_reduce_on_plateau
Network type: HybridDeepSetNetwork
LR Schedule: reduce_on_plateau


In [ ]:
# --- Step 4: Training the Classifier ---
print("\n--- Step 4: Training the Classifier ---")

loaded_simulator = ABCSimulator.load(simulator_path)
network = create_network_from_config(nre_config.network.to_dict())
estimator = NeuralRatioEstimator(
    network=network,
    training_config=nre_config.training,
    random_seed=nre_config.random_seed
)

key, train_key = jax.random.split(key)
training_history = estimator.train(
                                    key=train_key,
                                    simulator=loaded_simulator,
                                    output_dir=output_dir,
                                    num_epochs=nre_config.training.num_epochs,
                                    n_samples_per_epoch=nre_config.training.n_samples_per_epoch,
                                    batch_size=nre_config.training.batch_size
                                )


--- Step 4: Training the Classifier ---
⚠️  Warning: Could not load summary statistics: cannot unpack from multi-byte object
✅ Simulator loaded from: ../results/gauss_1D_simulator.yml
⚠️  Warning: Could not load summary statistics: cannot unpack from multi-byte object
✅ Simulator loaded from: ../results/gauss_1D_simulator.yml
🔧 Configured HybridDeepSetNetwork with data indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
🔧 Configured HybridDeepSetNetwork with data indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Initialized network with 37,297 parameters
✅ Auto-detected feature dimension: 12
Initialized network with 37,297 parameters
✅ Auto-detected feature dimension: 12
Epoch 1/50 | Train Loss: 0.3442, Val Loss: 0.3281, Train Acc 87.11%, Val Acc: 88.72%, Learning rate = 0.001000, Samples: 5120, Sims: 5120
Epoch 1/50 | Train Loss: 0.3442, Val Loss: 0.3281, Train Acc 87.11%, Val Acc: 88.72%, Learning rate = 0.001000, Samples: 5120, Sims: 5120
Epoch 2/50 | Train Loss: 0.2299, Val Loss: 0.2376, Train Acc 92.1

In [ ]:
# --- Step 5: Saving All Classifier Artifacts ---
print("\n--- Step 5: Saving All Classifier Artifacts ---")

final_config_path = save_classifier(
    estimator=estimator,
    simulator=loaded_simulator,
    output_dir=output_dir,
    filename_base="gauss_1D"
)


--- Step 5: Saving All Classifier Artifacts ---
✅ Simulator saved with hash: d1312ced323a
   - Configuration: ../results/gauss_1D_simulator_eps_inf_b7d408f5b633.yml
   - Observed Data: ../results/observed_data_eps_inf_d1312ced323a.npy
✅ Simulator saved to: ../results/gauss_1D_simulator_eps_inf_b7d408f5b633.yml
✅ Classifier saved to: ../results/gauss_1D_classifier_eps_inf_8066ecdd5fac.yml
   - Hash: 8066ecdd5fac (includes network + training + simulator + epsilon)
   - Associated simulator config: gauss_1D_simulator_eps_inf_b7d408f5b633.yml
